In [2]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk


In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
  ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
  ngrok.connect(5000)
  run_with_ngrok(app)


from flask import Flask, request, abort
from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    ImageMessage,
    VideoMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANEL_SECRET'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


# gemini part
import google.generativeai as genai
#輸入金鑰
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel("gemini-2.0-flash")
def ask_gemini(question):
  response = model.generate_content(question)
  return  response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'sticker':
            # 回覆貼圖
            reply = StickerMessage(
              package_id='446',
              sticker_id='1989'
            )
        elif action == 'image':
            # 回覆圖片
            reply = ImageMessage(
              original_content_url='https://scontent.ftpe19-1.fna.fbcdn.net/v/t39.30808-6/468659427_2394399047558495_6492302446918301419_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=6ee11a&_nc_ohc=LHsMKcsk5PQQ7kNvwE4td9F&_nc_oc=AdlJB_PROK-INLFCZMBVUjXkO497k9xmCbWvv_3ZkUYi27YJob5rH6LFXB2lLYypMW2YXrfbZoUfFi2bk81Hg6Df&_nc_zt=23&_nc_ht=scontent.ftpe19-1.fna&_nc_gid=X-5c91gmX8OrAUhajzgb7Q&oh=00_AfKtX7zUrnlWNygb7vL6w01MAPAjfkQ4_eaH-BMJqdL8bA&oe=681F9DE5',
              preview_image_url='https://scontent.ftpe19-1.fna.fbcdn.net/v/t39.30808-6/468659427_2394399047558495_6492302446918301419_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=6ee11a&_nc_ohc=LHsMKcsk5PQQ7kNvwE4td9F&_nc_oc=AdlJB_PROK-INLFCZMBVUjXkO497k9xmCbWvv_3ZkUYi27YJob5rH6LFXB2lLYypMW2YXrfbZoUfFi2bk81Hg6Df&_nc_zt=23&_nc_ht=scontent.ftpe19-1.fna&_nc_gid=X-5c91gmX8OrAUhajzgb7Q&oh=00_AfKtX7zUrnlWNygb7vL6w01MAPAjfkQ4_eaH-BMJqdL8bA&oe=681F9DE5'
            )
        elif action == 'video':
            # 回覆圖片
            reply = VideoMessage(
              original_content_url='https://youtu.be/AdkkF6MT0R0?si=VYMKUABV5IQvl8v0',
              preview_image_url='https://i.ytimg.com/vi/AdkkF6MT0R0/hqdefault.jpg?sqp=-oaymwFBCOADEI4CSFryq4qpAzMIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB8AEB-AH-BIAC4AOKAgwIABABGFkgZSgzMA8=&rs=AOn4CLBcZuqpD1Le_fu2bd8ddWp-nD2dHg'
            )
        elif action == 'location':
            reply = LocationMessage(
                title="Taipei 101",
                address="110台北市信義區信義路五段7號",
                latitude=25.034147572197465,
                longitude= 121.5645117282278
            )
        else:
            # 回覆文字
            response = ask_gemini(event.message.text)
            reply = TextMessage(text=response)

        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )

ngrok_start()
if __name__ == "__main__":
    app.run()